In [5]:
##%overwritefile
##%file:src/c_readtemplatefile.py
##%noruncode
    def readtemplatefile(self,filename,spacecount=0,*args: t.Any, **kwargs: t.Any):
        filecode=''
        newfilecode=''
        codelist1=None
        filenm=os.path.join(os.path.abspath(''),filename);
        if not os.path.exists(filenm):
            return filecode;
        template = self.jinja2_env.get_template(filenm)
        filecode=template.render(*args,**kwargs)

        for line in filecode.splitlines():
            if len(line)>0:
                for t in line:
                    newfilecode+=' '*spacecount + t+'\n'
        return newfilecode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_readtemplatefile.py created successfully


In [6]:
##%overwritefile
##%file:src/c_generate_file.py
##%noruncode
    def generate_file(self, source_filename, binary_filename, cflags=None, ldflags=None):

        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_generate_file.py created successfully


In [4]:
##%overwritefile
##%file:src/c_do_execute.py
##%noruncode
    def do_execute(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        self.silent = silent
        magics, code = self._filter_magics(code)
        
        if len(magics['replcmdmode'])>0:
            return self.send_replcmd(code, silent, store_history=True,
                   user_expressions=None, allow_stdin=False)
        
        ############# run gdb and send command begin
        if len(magics['rungdb'])>0:
            return self.replgdb_sendcmd(code,silent, store_history,
                   user_expressions, allow_stdin)
        ############# run gdb and send command
        #############send replcmd's command
        if magics['runmode']=='repl':
            if hasattr(self, 'replcmdwrapper'):
                if self.replcmdwrapper :
                    return self.repl_sendcmd(code, silent, store_history,
                        user_expressions, allow_stdin,magics)
        #FIXME:
        #############send replcmd's command end
        ############# only run command mark
        if len(magics['onlyruncmd'])>0:
            return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
        ############# only create source file
        if len(magics['onlycsfile'])<1 :
            magics, code = self._add_main(magics, code)
        # magics, code = self._add_main(magics, code)

        # replace stdio with wrapped version
        # headerDir = "\"" + self.resDir + "/stdio_wrap.h" + "\""
        # code = code.replace("<stdio.h>", headerDir)
        # code = code.replace("\"stdio.h\"", headerDir)

        with self.new_temp_file(suffix='.c') as source_file:
            source_file.write(code)
            source_file.flush()
            newsrcfilename=source_file.name 
            # Generate new src file
            
            if len(magics['file'])>0:
                newsrcfilename = self._fileshander(magics['file'],newsrcfilename,magics)
                # newsrcfilename = magics['file']
                # newsrcfilename = os.path.join(os.path.abspath(''),newsrcfilename)
                # if os.path.exists(newsrcfilename):
                #     newsrcfilename +=".new.c"
                # if not os.path.exists(os.path.dirname(newsrcfilename)) :
                #     os.makedirs(os.path.dirname(newsrcfilename))
                # os.rename(source_file.name,newsrcfilename)
                self._write_to_stdout("[C kernel] Info:file "+ newsrcfilename +" created successfully\n")

            if len(magics['onlycsfile'])>0:
                if len(magics['file'])<1:
                    self._write_to_stderr("[C kernel] Warning: no file name parameter\n")
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            # Generate executable file
            returncode,binary_filename=self._exec_gcc_(newsrcfilename,magics)
            if returncode!=0:
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
        ############# only run gcc，no not run executable file
        if len(magics['onlyrungcc'])>0:
            self._write_to_stderr("[C kernel] Info: only run gcc \n")
            return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
        
        ################# repl mode run code files
        #FIXME:
        if magics['runmode']=='repl':
            self._start_replprg(binary_filename,magics['args'],magics)
            return  {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}
        ############################################


        #################dynamically load and execute code
        #FIXME:
        if len(magics['dlrun'])>0:
            p = self.create_jupyter_subprocess([self.master_path, binary_filename] + magics['args'],env=magics['env'])
        ############################################
        else:
            p = self.create_jupyter_subprocess([binary_filename] + magics['args'],env=magics['env'])
        self.subprocess=p

        while p.poll() is None:
            p.write_contents(magics)

        p.write_contents(magics)
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()

        p.write_contents(magics)

        # now remove the files we have just created
        if(os.path.exists(source_file.name)):
            os.remove(source_file.name)
        # if(os.path.exists(binary_filename)):
            # os.remove(binary_filename)

        # if p.returncode != 0:
            # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_do_execute.py created successfully


In [8]:
##%overwritefile
##%file:src/c_filter_magics.py
##%noruncode
    def _filter_magics(self, code):

        magics = {'cflags': [],
                  'ldflags': [],
                  'file': [],
                  'include': '',
                  'templatefile': [],

                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                  'showpid': [],
                  'onlycsfile': [],
                  'onlyrungcc': [],
                  'onlyruncmd': [],
                  'dlrun': [],
                  'showinput': [],
                  'command': [],
                  'outputtype':'text/plain',
                  'env':None,
                  'runmode': [], #default real,interactive mode is repl
                  'rungdb': [],
                  'args': []}

        actualCode = ''
        for line in code.splitlines():
            orgline=line
            if line.strip().startswith('//%'):
                if line.strip()[3:] == "onlycsfile":
                    magics['onlycsfile'] += ['true']
                    continue
                elif line.strip()[3:] == "showpid":
                    magics['showpid'] += ['true']
                    continue
                elif line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                    continue
                elif line.strip()[3:] == "onlyrungcc":
                    magics['onlyrungcc'] += ['true']
                    continue
                elif line.strip()[3:] == "onlyruncmd":
                    magics['onlyruncmd'] += ['true']
                    continue
                elif line.strip()[3:] == "rungdb":
                    magics['rungdb'] += ['true']
                    continue
                elif line.strip()[3:] == "dlrun":
                    magics['dlrun'] += ['true']
                    continue
                elif line.strip()[3:] == "showinput":
                    magics['showinput'] += ['true']
                    continue

                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 2)
                key = key.strip().lower()

                if key in ['ldflags', 'cflags']:
                    for flag in value.split():
                        magics[key] += [flag]
                elif key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "file":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='newfile'
                elif key == "include":
                    if len(value)>0:
                        magics[key] = value
                    else:
                        magics[key] =''
                        continue
                    if len(magics['include'])>0:
                        index1=line.find('//%')
                        line=self.readcodefile(magics['include'],index1)
                        actualCode += line + '\n'
                elif key == "templatefile":
                    index1=line.find('//%')
                    if len(value)>0:
                        magics[key] =value.split(" ",1)
                    else:
                        magics[key] =None
                        continue
                    templatefile=magics['templatefile'][0]
                    if len(magics['templatefile'])>1:
                        argsstr=magics['templatefile'][1]
                        templateargsdict=_filter_dict(argsstr)
                    else:
                        templateargsdict=None
                    if len(magics['templatefile'])>0:
                        line=self.readtemplatefile(templatefile,index1,templateargsdict)
                        actualCode += line + '\n'
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.send_cmd(pid=pid,cmd=cmd)
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value
                elif key == "command":
                    # for flag in value.split():
                    magics[key] = [value]
                    if len(magics['command'])>0:
                        self.do_shell_command(magics['command'],env=magics['env'])
                elif key == "env":
                    envdict=self._filter_env(value)
                    magics[key] =dict(envdict)
                elif key == "outputtype":
                    magics[key]=value
                elif key == "args":
                    # Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]

                # always add empty line, so line numbers don't change
                # actualCode += '\n'

            # keep lines which did not contain magics
            else:
                actualCode += line + '\n'
        newactualCode=actualCode
        if len(magics['file'])>0 and len(magics['noruncode'])>0:
            newactualCode=''
            for line in actualCode.splitlines():
                line=cleantestcode(line)
                newactualCode += line
        return magics, actualCode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_filter_magics.py created successfully


In [9]:
##%overwritefile
##%file:src/compile_with_gcc.py
##%noruncode
    def compile_with_gcc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-Wdeclaration-after-statement', '-Wvla', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=iso9899:199409', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c11', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        outfile=binary_filename
        if self.linkMaths:
            cflags = cflags + ['-lm']
        if self.wError:
            cflags = cflags + ['-Werror']
        if self.wAll:
            cflags = cflags + ['-Wall']
        if self.readOnlyFileSystem:
            cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        if self.bufferedOutput:
            cflags = ['-DBUFFERED_OUTPUT'] + cflags

        for s in cflags:
                if s.startswith('-o'):
                    if(len(s)>2):
                        outfile=s[2:]
                    else:
                        outfile=cflags[cflags.index('-o')+1]
                        if outfile.startswith('-'):
                            outfile=binary_filename
                binary_filename=outfile
        args = ['gcc', source_filename] + ['-o', binary_filename]+ cflags  + ldflags
        self._write_to_stdout(''.join((' '+ str(s) for s in args))+"\n")
        return self.create_jupyter_subprocess(args,env=env),binary_filename,args


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/compile_with_gcc.py created successfully


In [10]:
##%overwritefile
##%file:src/c_exec_gcc_.py
##%noruncode
    def _exec_gcc_(self,source_filename,magics):
        self._write_to_stdout('Generating executable file\n')
        with self.new_temp_file(suffix='.out') as binary_file:
            p,outfile,gcccmd = self.compile_with_gcc(
                source_filename, 
                binary_file.name, 
                magics['cflags'], 
                magics['ldflags'],
                magics['env'])
            while p.poll() is None:
                p.write_contents()
            p.write_contents()
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if p.returncode != 0:  # Compilation failed
                self._write_to_stdout(''.join(('Error: '+ str(s) for s in gcccmd))+"\n")
                self._write_to_stderr("[C kernel] GCC exited with code {}, the executable will not be executed".format(p.returncode))

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_exec_gcc_.py created successfully


In [11]:
##%overwritefile
##%file:src/c_start_gdb.py
##%noruncode
    def _start_gdb(self):
        # Signal handlers are inherited by forked processes, and we can't easily
        # reset it from the subprocess. Since kernelapp ignores SIGINT except in
        # message handlers, we need to temporarily reset the SIGINT handler here
        # so that bash and its children are interruptible.
        sig = signal.signal(signal.SIGINT, signal.SIG_DFL)
        try:
            if hasattr(self, 'gdbwrapper'):
                return
        finally:
            pass
        try:
            # self._write_to_stdout("------exec gdb-----\n")
            child = pexpect.spawn('gdb', ['-q'], echo=False,encoding='utf-8')
            self.gdbwrapper = IREPLWrapper(child, u'(gdb)', prompt_change=None,
                                            extra_init_cmd='set pagination off',
                                            line_output_callback=self.process_output)
        except Exception as e:
            # self._write_to_stdout("-----------IREPLWrapper err "+str(e)+"!\n")
            exitcode = 1
        finally:
            signal.signal(signal.SIGINT, sig)


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_start_gdb.py created successfully


In [12]:
##%overwritefile
##%file:src/c_do_replexecutegdb.py
##%noruncode
    def do_replexecutegdb(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        self.silent = silent
        if not code.strip():
            return {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}

        interrupted = False
        try:
            # Note: timeout=None tells IREPLWrapper to do incremental
            # output.  Also note that the return value from
            # run_command is not needed, because the output was
            # already sent by IREPLWrapper.
            self.gdbwrapper.run_command(code.rstrip(), timeout=None)
        except KeyboardInterrupt:
            self.gdbwrapper.child.sendintr()
            interrupted = True
            self.gdbwrapper._expect_prompt()
            output = self.gdbwrapper.child.before
            self.process_output(output)
        except EOF:
            output = self.gdbwrapper.child.before + 'Restarting GDB'
            self._start_gdb()
            self.process_output(output)

        if interrupted:
            return {'status': 'abort', 'execution_count': self.execution_count}

        try:
            if code.rstrip().startswith('shell'):
                exitcode = int(self.gdbwrapper.run_command('shell echo $?').rstrip())
            else:
                exitcode = int(self.gdbwrapper.run_command('echo $?').rstrip())
        except Exception:
            exitcode = 1

        if exitcode:
            error_content = {'execution_count': self.execution_count,
                             'ename': '', 'evalue': str(exitcode), 'traceback': []}

            self.send_response(self.iopub_socket, 'error', error_content)
            error_content['status'] = 'error'
            return error_content
        else:
            return {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_do_replexecutegdb.py created successfully


In [13]:
##%overwritefile
##%file:src/c_replgdb_sendcmd.py
##%noruncode
    def replgdb_sendcmd(self,code,silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        self._start_gdb()
        return self.do_replexecutegdb( code.replace('//%rungdb', ''), silent, store_history,
                   user_expressions, False)


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_replgdb_sendcmd.py created successfully


In [4]:
##%overwritefile
##%file:../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py
###%file:c_kernel.py
##%noruncode
#
#   MyC Jupyter Kernel 
#   generated by MyPython
#
##%include:src/head.py

##%include:src/common.py

##%include:src/IREPLWrapper.py
##%include:src/RealTimeSubprocess.py

class CKernel(Kernel):
    implementation = 'jupyter-MyC-kernel'
    implementation_version = '1.0'
    language = 'C'
    language_version = 'C11'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.c'}
    banner = "C kernel.\n" \
             "Uses gcc, compiles in C11, and creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"

##%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(CKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyC Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.files = []
        mastertemp = tempfile.mkstemp(suffix='.out')
        os.close(mastertemp[0])
        self.master_path = mastertemp[1]
        self.resDir = path.join(path.dirname(path.realpath(__file__)), 'resources')
        filepath = path.join(self.resDir, 'master.c')
        subprocess.call(['gcc', filepath, '-std=c11', '-rdynamic', '-ldl', '-o', self.master_path])

        self.chk_replexit_thread = Thread(target=self.chk_replexit, args=(self.g_rtsps))
        self.chk_replexit_thread.daemon = True
        self.chk_replexit_thread.start()
        self.init_plugin()
    
##%include:src/_templateHander.py
##%include:src/_readtemplatefile.py

##%include:src/cleannotes.py
##%include:src/kernel_common.py
##%include:src/_is_specialID.py

##%include:src/repl_listpid.py
##%include:src/chk_replexit.py
##%include:src/cleanup_files.py
##%include:src/new_temp_file.py
##%include:src/log_output.py
##%include:src/_write_display_data.py
##%include:src/_write_to_stdout.py
##%include:src/_write_to_stderr.py
##%include:src/_read_from_stdin.py
##%include:src/readcodefile.py
    #####################################################################
##%include:src/_start_replprg.py
##%include:src/process_output.py
##%include:src/send_replcmd.py
    #####################################################################
##%include:src/do_shell_command.py

##%include:src/create_jupyter_subprocess.py
##%include:src/c_generate_file.py

##%include:src/compile_with_gcc.py
##%include:src/_filter_env.py
#####################################################################
##%include:src/c_filter_magics.py
#####################################################################

##%include:src/_add_main.py

##%include:src/c_exec_gcc_.py
##%include:src/c_start_gdb.py
##%include:src/c_do_replexecutegdb.py
##%include:src/c_replgdb_sendcmd.py

##%include:src/c_do_execute.py
##%include:src/do_shutdown.py
#####################################################################
##%include:src/pluginmng.py


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py created successfully
